# HW1 - Notebook: Analytics in PySPARK

Il notebook mostra una serie di Analytics sui progetti fatti negli anni passati e in corso dall'Università di Napoli, Federico II

Questo notebook è scritto in **Python**

### 1. Installazione di Spark

Scarica e installa Apache Spark 3.2.4, insieme alle sue dipendenze (Java Development Kit 8+).

In [ ]:
# Installa Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Scarica Spark 3.2.4
!wget -q -N https://apache.osuosl.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz 

# Decomprimo l'archivio di Spark
!tar xf spark-3.2.4-bin-hadoop3.2.tgz

# Installa findspark 
!pip install -q findspark

Si impostano le variabili d'ambiente per rendere visibile il runtime Spark al sistema operativo Linux. Si noti che è possibile gestire più versioni di Spark puntando alla versione corretta attraverso le variabili d'ambiente. Si esegue la seguente serie di comandi per puntare alla versione di Apache Spark 3.2.4 scaricata in precedenza.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

Si verifica che l'installazione sia andata a buon fine. Si **importa** la libreria `findspark` che consente di trovare e inizializzare automaticamente la configurazione di Spark senza dover configurare manualmente variabili di ambiente e altre impostazioni.

In [ ]:
import findspark
findspark.init()

In [ ]:
# Si verifica la versione di Spark in esecuzione sul cluster virtuale
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert  "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

Se l'ambiente non è configurato correttamente, l'istruzione mostrerà la versione di Spark che non sarà la 3.0 o successiva.

In [ ]:
print("Spark version:", sc.version)

Spark version: 3.2.4


In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### 2. Caricamento del Dataset su Spark
L'RDD (Resilient Distributed Dataset) è la rappresentazione dei dati alla base di Spark. Tuttavia, scrivere codice funzionale non è sempre facile e si è deciso di aggiungere un modello di dati più *responsive*, basato sui concetti principali di **tabelle di dati** e **istruzioni SQL**, il cosiddetto **Spark SQL**.

Spark SQL, a partire da Spark v1.3, offre all'utente approfondimenti su dati come **dataset** (fino a Spark v2.0 chiamati DataFrame). Un dataset è un oggetto organizzato come una tabella: ogni colonna ha un nome e un tipo, e ogni riga è una combinazione dei valori disponibili delle colonne.

Come gli RDD, anche i dataset sono oggetti immutabili: lo sviluppatore può trasformare il dataset in un altro utilizzando operazioni *simili a SQL*.

Il motore SQL di Spark analizza le operazioni **'SQL-like'** applicate al dataset e traduce le operazioni su RDD, esegue le operazioni sull'RDD e restituisce l'RDD in formato dataset.

L'interazione con gli oggetti SQL di Spark viene eseguita dalla `SparkSession` (prima di Spark v2.0 veniva utilizzato `SQLContext`).

In [ ]:
# Librerie necessarie per l'esecuzione corretta delle Analytics
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import split, explode, trim, count, sum, col, current_date, lower, regexp_replace
import time

spark = SparkSession(sc)
print(spark)

Spark SQL lavora nativamente con i file CSV ed è in grado di identificare automaticamente il nome e il tipo di dati di ogni colonna. Essendo che il dataset fornito è in formato .xlsx, occorre innanzitutto convertirlo in formato .csv.

In [ ]:
# Converte il file da .xlsx a .csv
import pandas as pd
from traitlets.traitlets import Int

# Scarica il dataset dal repo GitHub
!wget -q -N https://github.com/giuseppericcio/BigData/raw/main/Datasets/Dataset_Projects_Unina.xlsx
!wget -q -N https://github.com/giuseppericcio/BigData/raw/main/Datasets/stopwords.txt
!mkdir risultati

# Lettura del file .xlsx
read_file = pd.read_excel('/content/Dataset_Projects_Unina.xlsx', dtype = {'Funding_Amount_in_EUR' : Int, 
                                                                           'Start_Year' : Int, 
                                                                           'End_Year' : Int})
# Conversione in .csv
read_file.to_csv('/content/Dataset_Projects_Unina.csv', index = None, header=True)

In [ ]:
# Legge il file .csv dal filesystem

# Identifica il tipo per ogni colonna (float, integer, string)
# Prende i nomi delle colonne dalla prima riga del CSV
projectsDS = spark.read \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("multiLine", "true") \
  .csv('/content/Dataset_Projects_Unina.csv')

# Stampa a video lo Schema del dataset, che mostra i nomi e i tipi di colonne
projectsDS.printSchema()

root
 |-- Title_translated: string (nullable = true)
 |-- Abstract_translated: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- Funding_Amount_in_EUR: integer (nullable = true)
 |-- Start_Date: string (nullable = true)
 |-- Start_Year: integer (nullable = true)
 |-- End_Date: string (nullable = true)
 |-- End_Year: integer (nullable = true)
 |-- Researchers: string (nullable = true)
 |-- Research_Organization_original: string (nullable = true)
 |-- Research_Organization_standardized: string (nullable = true)
 |-- City_of_Research_organization: string (nullable = true)
 |-- Country_of_Research_organization: string (nullable = true)
 |-- Funder: string (nullable = true)
 |-- Funder_Group: string (nullable = true)
 |-- Funder_Country: string (nullable = true)
 |-- Program: string (nullable = true)
 |-- Fields_of_Research_ANZSRC_2020: string (nullable = true)
 |-- RCDC_Categories: string (nullable = true)
 |-- HRCS_HC_Categories: string (nullable = true)
 |-- HRCS_RAC_

In [ ]:
# Il comando 'show(10)' mostra le prime 10 righe del dataset. Per impostazione predefinita, senza parametri, mostra le prime 20 righe
projectsDS.show(10)

+--------------------+--------------------+--------+---------------------+----------+----------+----------+--------+-----------+------------------------------+----------------------------------+-----------------------------+--------------------------------+-------------------+--------------------+--------------+--------------------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------------+
|    Title_translated| Abstract_translated|Keywords|Funding_Amount_in_EUR|Start_Date|Start_Year|  End_Date|End_Year|Researchers|Research_Organization_original|Research_Organization_standardized|City_of_Research_organization|Country_of_Research_organization|             Funder|        Funder_Group|Funder_Country|             Program|Fields_of_Research_ANZSRC_2020|     RCDC_Categories|  HRCS_HC_Categories| HRCS_RAC_Categories|        Cancer_Types|      CSO_Categories| Units_o

# Analytics descrittive

In [ ]:
# Calcolare la somma finanziata tra i 5 dipartimenti con più progetti.
# Trovo i 5 Dipartimenti con più progetti.
Top5Dip = projectsDS.select(explode(split(projectsDS["Units_of_Assessment"], "; ")).alias("Dipartimento"), projectsDS["Funding_Amount_in_EUR"]) \
          .filter(col("Dipartimento").isNotNull()) \
          .groupBy(col("Dipartimento")) \
          .count() \
          .orderBy(col("count").desc()) \
          .limit(5)

# Somma gli importi finanziati per tutti i progetti riguardanti i 5 Dipartimenti trovati.
Top5DipSommaProgetti = projectsDS.select(explode(split(projectsDS["Units_of_Assessment"], "; ")).alias("Dipartimento"), projectsDS["Funding_Amount_in_EUR"]) \
                      .filter(col("Dipartimento").isNotNull()) \
                      .join(Top5Dip, "Dipartimento") \
                      .groupBy(col("Dipartimento")) \
                      .agg(F.sum("Funding_Amount_in_EUR").alias("Somma Finanziata (EUR)")) \
                      .orderBy(col("Somma Finanziata (EUR)").desc())

# Mostra i risultati
Top5DipSommaProgetti.show(truncate = False)

# Salva il risultato in un file CSV
risultato = Top5DipSommaProgetti.toPandas()
risultato.to_csv('/content/risultati/Top5DipSommaProgetti.csv')

+--------------------------------------------------------------+----------------------+
|Dipartimento                                                  |Somma Finanziata (EUR)|
+--------------------------------------------------------------+----------------------+
|B12 Engineering                                               |1101742946            |
|B11 Computer Science and Informatics                          |488879107             |
|A06 Agriculture, Veterinary and Food Science                  |190442745             |
|A01 Clinical Medicine                                         |177342410             |
|A03 Allied Health Professions, Dentistry, Nursing and Pharmacy|87311094              |
+--------------------------------------------------------------+----------------------+



In [ ]:
# Le 10 tematiche più trattate negli ultimi 10 anni.
Tem_Top10 = projectsDS.select(explode(split(projectsDS["Fields_of_Research_ANZSRC_2020"], "; ")).alias("Ambito di Ricerca"), "End_Year") \
            .filter(col("End_Year") >= 2012 ) \
            .filter(col("Ambito di Ricerca").isNotNull()) \
            .groupBy(col("Ambito di Ricerca")) \
            .count() \
            .select("Ambito di Ricerca", col("count").alias("Numero progetti")) \
            .orderBy(col("Numero progetti").desc()) \
            .limit(10)

# Mostra i risultati
Tem_Top10.show(truncate = False)

# Salva il risultato in un file CSV
risultato = Tem_Top10.toPandas()
risultato.to_csv('/content/risultati/Tem_Top10.csv')

+---------------------------------------------+---------------+
|Ambito di Ricerca                            |Numero progetti|
+---------------------------------------------+---------------+
|32 Biomedical and Clinical Sciences          |187            |
|40 Engineering                               |150            |
|31 Biological Sciences                       |143            |
|34 Chemical Sciences                         |78             |
|46 Information and Computing Sciences        |68             |
|3101 Biochemistry and Cell Biology           |52             |
|3211 Oncology and Carcinogenesis             |46             |
|30 Agricultural, Veterinary and Food Sciences|45             |
|44 Human Society                             |40             |
|37 Earth Sciences                            |38             |
+---------------------------------------------+---------------+



In [ ]:
# Le 100 istituzioni con cui ha collaborato di più (o meno) la Federico II.
Istituzioni = projectsDS.select(explode(split("Research_Organization_original", "; ")).alias("Istituzione")) \
              .filter(col("Istituzione") != 'University of Naples Federico II') \
              .groupBy(trim("Istituzione").alias("Istituzione")) \
              .count() \
              .select("Istituzione", col("count").alias("Numero_Collaborazioni")) \
              .orderBy(col("Numero_Collaborazioni").desc()) \
              .limit(100)

# Mostra i risultati
Istituzioni.show(truncate = False)

# Salva il risultato in un file CSV
risultato = Istituzioni.toPandas()
risultato.to_csv('/content/risultati/Istituzioni.csv')

+---------------------------------------+---------------------+
|Istituzione                            |Numero_Collaborazioni|
+---------------------------------------+---------------------+
|Sapienza University of Rome            |476                  |
|University of Milan                    |350                  |
|University of Florence                 |346                  |
|University of Padua                    |322                  |
|University of Bologna                  |317                  |
|University of Turin                    |290                  |
|University of Pisa                     |290                  |
|University of Campania Luigi Vanvitelli|280                  |
|National Research Council              |255                  |
|University of Catania                  |243                  |
|University of Palermo                  |242                  |
|University of Genoa                    |234                  |
|University of Bari Aldo Moro           

In [ ]:
# TOP 10 dipartimenti con progetti ancora in corso
Dipartimenti = projectsDS.filter(col("End_Date") > current_date()) \
              .select(explode(split("Units_of_Assessment", "; ")).alias("Dipartimento")) \
              .groupBy(trim("Dipartimento").alias("Dipartimento")) \
              .count() \
              .select("Dipartimento", col("count").alias("Numero_Progetti_In_Corso")) \
              .orderBy(col("Numero_Progetti_In_Corso").desc()) \
              .limit(10)

# Mostra i risultati
Dipartimenti.show(truncate = False)

# Salva il risultato in un file CSV
risultato = Dipartimenti.toPandas()
risultato.to_csv('/content/risultati/Dipartimenti.csv')

+--------------------------------------------------------------+------------------------+
|Dipartimento                                                  |Numero_Progetti_In_Corso|
+--------------------------------------------------------------+------------------------+
|B12 Engineering                                               |28                      |
|B11 Computer Science and Informatics                          |14                      |
|A01 Clinical Medicine                                         |12                      |
|A06 Agriculture, Veterinary and Food Science                  |11                      |
|C14 Geography and Environmental Studies                       |10                      |
|A03 Allied Health Professions, Dentistry, Nursing and Pharmacy|7                       |
|C17 Business and Management Studies                           |6                       |
|B07 Earth Systems and Environmental Sciences                  |3                       |
|A05 Biolo

In [ ]:
# Somma Finanziata dalla Federico II nelle ricerche negli ultimi 20 anni
SommaFinanziata = projectsDS.filter(col("Start_Year") > '2001') \
                  .groupBy(col("Start_Year")) \
                  .agg(F.sum("Funding_Amount_in_EUR").alias("Somma_Finanziata_EUR")) \
                  .orderBy(col("Start_Year").desc())

# Mostra i risultati
SommaFinanziata.show(truncate = False)

# Salva il risultato in un file CSV
risultato = SommaFinanziata.toPandas()
risultato.to_csv('/content/risultati/SommaFinanziata.csv')

+----------+--------------------+
|Start_Year|Somma_Finanziata_EUR|
+----------+--------------------+
|2023      |122006193           |
|2022      |314043199           |
|2021      |102775511           |
|2020      |412702370           |
|2019      |63059973            |
|2018      |184665016           |
|2017      |222371781           |
|2016      |91142273            |
|2015      |65034430            |
|2014      |327815115           |
|2013      |78567634            |
|2012      |77543183            |
|2011      |93344945            |
|2010      |108964206           |
|2009      |103549195           |
|2008      |167363184           |
|2007      |102475007           |
|2006      |168668207           |
|2005      |85702511            |
|2004      |172466365           |
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Somma e numero progetti della federico II nel 2020.
ProgFondi2020 = projectsDS.filter(col("Start_Year") == '2020') \
                .select(explode(split("Fields_of_Research_ANZSRC_2020", "; ")).alias("Campi_di_ricerca"), "Funding_Amount_in_EUR") \
                .groupBy(col("Campi_di_ricerca")) \
                .agg(count("Campi_di_ricerca").alias("Numero_progetti"), F.sum("Funding_Amount_in_EUR").alias("Somma_Finanziata_Totale")) \
                .orderBy(col("Numero_progetti").desc())

# Mostra i risultati
ProgFondi2020.show(truncate = False)

# Salva il risultato in un file CSV
risultato = ProgFondi2020.toPandas()
risultato.to_csv('/content/risultati/ProgFondi2020.csv')

+---------------------------------------------+---------------+-----------------------+
|Campi_di_ricerca                             |Numero_progetti|Somma_Finanziata_Totale|
+---------------------------------------------+---------------+-----------------------+
|32 Biomedical and Clinical Sciences          |32             |29903187               |
|31 Biological Sciences                       |25             |15639113               |
|3211 Oncology and Carcinogenesis             |17             |15737406               |
|40 Engineering                               |16             |172178921              |
|34 Chemical Sciences                         |10             |7570303                |
|3101 Biochemistry and Cell Biology           |9              |2883287                |
|46 Information and Computing Sciences        |7              |165248297              |
|44 Human Society                             |6              |10151596               |
|51 Physical Sciences           

In [ ]:
# Somma Finanziata dalla Federico II nelle ricerche sul Cancro negli ultimi 20 anni.
SommaFinanziataCancro = projectsDS.filter(col("Cancer_Types").isNotNull()) \
                        .filter(col("Start_Year") > '2001') \
                        .groupBy(col("Start_Year")) \
                        .agg(F.sum("Funding_Amount_in_EUR").alias("Somma_Finanziata_Cancro_EUR")) \
                        .orderBy(col("Start_Year").desc())

# Mostra i risultati
SommaFinanziataCancro.show(truncate = False)

# Salva il risultato in un file CSV
risultato = SommaFinanziataCancro.toPandas()
risultato.to_csv('/content/risultati/SommaFinanziataCancro.csv')

+----------+---------------------------+
|Start_Year|Somma_Finanziata_Cancro_EUR|
+----------+---------------------------+
|2023      |8852273                    |
|2022      |4099002                    |
|2021      |4595975                    |
|2020      |17067493                   |
|2017      |17031248                   |
|2016      |998050                     |
|2015      |4707851                    |
|2014      |759100                     |
|2013      |13305339                   |
|2012      |801647                     |
|2010      |2515122                    |
|2009      |10746439                   |
|2008      |5424964                    |
|2007      |22189172                   |
|2006      |9319903                    |
|2005      |7124936                    |
|2004      |6607273                    |
|2003      |10003159                   |
|2002      |19902476                   |
+----------+---------------------------+



In [ ]:
# Le parole chiavi più utilizzate nei titoli dei progetti.

start_time = time.time()

# Carica il file di stopwords come RDD
stopwords_rdd = sc.textFile('/content/stopwords.txt')

# Converti l'RDD in DataFrame
stopwords_df = stopwords_rdd.map(lambda x: (x, )).toDF(['word'])

# Seleziona solo la colonna 'word'
stopwords = stopwords_df.select(col('word'))

# Esegui la query
parolechiavi = projectsDS.select(explode(split(regexp_replace(lower(col("Title_translated")), '[\\p{Punct},\\p{Cntrl}]', ''), ' ')).alias("Topic_Titolo")) \
              .join(stopwords, col("Topic_Titolo") == col("word"), "left_anti") \
              .filter(col("Topic_Titolo") != '') \
              .groupBy(trim("Topic_Titolo").alias("Topic_Titolo")) \
              .count() \
              .select("Topic_Titolo", col("count").alias("Conteggio")) \
              .orderBy(col("Conteggio").desc(), col("Topic_Titolo").asc()) \
              .limit(100)

# Mostra i risultati
parolechiavi.show(truncate = False)
end_time = time.time()


execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Salva il risultato in un file CSV
risultato = parolechiavi.toPandas()
risultato.to_csv('/content/risultati/parolechiavi.csv')

+----------------+---------+
|Topic_Titolo    |Conteggio|
+----------------+---------+
|molecular       |185      |
|systems         |172      |
|development     |138      |
|study           |129      |
|mechanisms      |114      |
|control         |104      |
|innovative      |102      |
|european        |99       |
|role            |95       |
|analysis        |87       |
|disease         |76       |
|design          |75       |
|applications    |71       |
|processes       |70       |
|italian         |69       |
|models          |69       |
|integrated      |68       |
|italy           |66       |
|characterization|62       |
|functional      |60       |
+----------------+---------+
only showing top 20 rows

Execution time: 2.771289825439453 seconds


In [ ]:
# I 10 Paesi esteri con cui ha maggiormente collaborato l'Università.
top10paesi = projectsDS.select(explode(split("Country_of_Research_organization", "; ")).alias("Paese"), "Title_translated") \
            .distinct() \
            .filter(col("Paese") != 'Italy') \
            .groupBy(trim("Paese").alias("Paese")) \
            .count() \
            .select("Paese", col("count").alias("Numero_Collaborazioni")) \
            .orderBy(col("Numero_Collaborazioni").desc()) \
            .limit(10)

# Mostra i risultati
top10paesi.show(truncate = False)

# Salva il risultato in un file CSV
risultato = top10paesi.toPandas()
risultato.to_csv('/content/risultati/top10paesi.csv')

+--------------+---------------------+
|Paese         |Numero_Collaborazioni|
+--------------+---------------------+
|United Kingdom|414                  |
|Germany       |385                  |
|France        |332                  |
|Spain         |262                  |
|Netherlands   |241                  |
|Belgium       |193                  |
|Sweden        |149                  |
|Greece        |138                  |
|United States |135                  |
|Switzerland   |124                  |
+--------------+---------------------+



In [ ]:
# Gli ambiti di ricerca con progetti di maggiore durata.
start_time = time.time()
ricercadurata = projectsDS.select(explode(split("Units_of_Assessment", ";")).alias("Dipartimento"), "End_Year", "Start_Year", "Title_translated") \
                .groupBy(col("Title_translated").alias("Titolo_Progetto"), trim("Dipartimento").alias("Dipartimento")) \
                .agg((F.max(col("End_Year")) - F.min(col("Start_Year"))).alias("Durata_Progetto_in_Anni")) \
                .orderBy(col("Durata_Progetto_in_Anni").desc()) \
                .limit(10)

# Mostra i risultati
ricercadurata.show(truncate = False)
end_time = time.time()

elapsed_time = end_time - start_time
print("Tempo di esecuzione:", elapsed_time, "secondi")

# Salva il risultato in un file CSV
risultato = ricercadurata.toPandas()
risultato.to_csv('/content/risultati/ricercadurata.csv')

+-----------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------+-----------------------+
|Titolo_Progetto                                                                                                        |Dipartimento                                                  |Durata_Progetto_in_Anni|
+-----------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------+-----------------------+
|The role of p53/p73 family members in the cytotoxic response.                                                          |A01 Clinical Medicine                                         |16                     |
|Mechanisms of Hepatitis C Virus Infection and Pathogenesis                                                             |A01 Clinical Medicine                      

In [ ]:

# Capacità di gestione delle risorse finanziare dei progetti negli ultimi 20 anni. (GRAFICO CON PYTHON)
start_time = time.time()
capacitàfondi = projectsDS.groupBy("Start_Year") \
                .agg(F.sum("Funding_Amount_in_EUR").alias("Somma_Finanziata_EUR"), F.count("*").alias("Numero_Progetti")) \
                .filter(col("Start_Year").isNotNull()) \
                .groupBy(col("Start_Year"), col("Somma_Finanziata_EUR"), col("Numero_Progetti")) \
                .agg((col("Numero_Progetti") / (col("Somma_Finanziata_EUR"))).alias("Capacita_Progetti"), 
                    ((col("Somma_Finanziata_EUR")) / col("Numero_Progetti")).alias("Fondi_media_Progetti")) \
                .select(col("Start_Year").alias("Anno_Inizio"), "Capacita_Progetti", "Fondi_media_Progetti") \
                .orderBy(col("Anno_Inizio").desc()) \
                .limit(22)


# Mostra i risultati

capacitàfondi.show(truncate = False)
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")



# Salva il risultato in un file CSV
risultato = capacitàfondi.toPandas()
risultato.to_csv('/content/risultati/capacitàfondi.csv')



+-----------+---------------------+--------------------+
|Anno_Inizio|Capacita_Progetti    |Fondi_media_Progetti|
+-----------+---------------------+--------------------+
|2023       |1.4753349446777672E-7|6778121.833333333   |
|2022       |1.1781818589868586E-7|8487654.027027028   |
|2021       |3.502779956988003E-7 |2854875.3055555555  |
|2020       |2.5926674470030303E-7|3857031.495327103   |
|2019       |3.1715839776842276E-7|3152998.65          |
|2018       |1.4079548234517793E-7|7102500.615384615   |
|2017       |8.769098269712558E-7 |1140368.1076923078  |
|2016       |3.4012757175805785E-7|2940073.3225806453  |
|2015       |1.1993647057412511E-6|833774.7435897436   |
|2014       |7.626249936644929E-8 |1.31126046E7        |
|2013       |3.181971853702506E-7 |3142705.36          |
|2012       |6.190099263786993E-7 |1615482.9791666667  |
|2011       |3.321015401530313E-7 |3011127.2580645164  |
|2010       |7.892500038040015E-7 |1267025.6511627906  |
|2009       |1.0236680256181615

In [ ]:
#Numero di progetti di ricerca per ogni obiettivo di sviluppo sostenibile.
ProgettiFinanziatiSostenibilità = projectsDS.select(explode(split("Sustainable_Development_Goals", "; ")).alias("Obiettivo_Sostenibilita"), "Funding_Amount_in_EUR") \
                                  .filter(col("Obiettivo_Sostenibilita").isNotNull()) \
                                  .groupBy(col("Obiettivo_Sostenibilita")) \
                                  .agg(F.count("*").alias("Numero_Progetti"), F.sum("Funding_Amount_in_EUR").alias("Somma_finanziata")) \
                                  .select(trim("Obiettivo_Sostenibilita").alias("Obiettivo_Sostenibilita"), "Numero_Progetti", "Somma_finanziata") \
                                  .orderBy(col("Numero_Progetti").desc())

# Mostra i risultati
ProgettiFinanziatiSostenibilità.show(truncate = False)

# Salva il risultato in un file CSV
risultato = ProgettiFinanziatiSostenibilità.toPandas()
risultato.to_csv('/content/risultati/ProgettiFinanziatiSostenibilità.csv')

+-----------------------------------------+---------------+----------------+
|Obiettivo_Sostenibilita                  |Numero_Progetti|Somma_finanziata|
+-----------------------------------------+---------------+----------------+
|7 Affordable and Clean Energy            |82             |311321685       |
|13 Climate Action                        |53             |139135505       |
|11 Sustainable Cities and Communities    |40             |318092831       |
|3 Good Health and Well Being             |34             |98917877        |
|12 Responsible Consumption and Production|25             |199551657       |
|16 Peace, Justice and Strong Institutions|23             |7537352         |
|2 Zero Hunger                            |21             |90328741        |
|4 Quality Education                      |15             |15812081        |
|9 Industry, Innovation and Infrastructure|13             |298867713       |
|10 Reduced Inequalities                  |9              |4224395         |

In [ ]:
#Numero di progetti di ricerca per ogni tipo di cancro.
ProgettiCancro = projectsDS.select(explode(split("Cancer_types", "; ")).alias("Tipo_Cancro")) \
                .filter(col("Tipo_Cancro").isNotNull()) \
                .filter(col("Tipo_Cancro") != 'Not Site-Specific Cancer') \
                .groupBy(col("Tipo_Cancro")) \
                .count() \
                .select(trim("Tipo_Cancro").alias("Tipo_Cancro"), col("count").alias("Numero_Progetti")) \
                .orderBy(col("Numero_Progetti").desc()) \
                .limit(25)

# Mostra i risultati
ProgettiCancro.show(truncate = False)

# Salva il risultato in un file CSV
risultato = ProgettiCancro.toPandas()
risultato.to_csv('/content/risultati/ProgettiCancro.csv')

+-------------------------------+---------------+
|Tipo_Cancro                    |Numero_Progetti|
+-------------------------------+---------------+
|Thyroid Cancer                 |23             |
|Breast Cancer                  |22             |
|Liver Cancer                   |19             |
|Colon and Rectal Cancer        |14             |
|Brain Tumor                    |8              |
|Stomach Cancer                 |8              |
|Oral Cavity and Lip Cancer     |7              |
|Lung Cancer                    |7              |
|Leukemia / Leukaemia           |7              |
|Esophageal / Oesophageal Cancer|6              |
|Pancreatic Cancer              |5              |
|Skin Cancer                    |5              |
|Head and Neck Cancer           |4              |
|Pituitary Tumor                |4              |
|Bladder Cancer                 |4              |
|Melanoma                       |3              |
|Nervous System                 |3              |


In [ ]:
#Tipi di malattie, possibili cure e gli obiettivi perseguiti.
Settori_Medici = projectsDS.select(explode(split(col("RCDC_Categories"), "; ")).alias("Cat_Medica"), col("Funding_Amount_in_EUR")) \
                .filter(col("Cat_Medica").isNotNull()) \
                .groupBy(trim(col("Cat_Medica")).alias("Settore_Medico")) \
                .agg(F.count("*").alias("Numero_Progetti"), F.sum("Funding_Amount_in_EUR").alias("Somma_Finanziata_EUR")) \
                .orderBy(col("Somma_Finanziata_EUR").desc(), col("Numero_Progetti").desc()) \
                .limit(10)

# Mostra i risultati
Settori_Medici.show(truncate = False)

# Salva il risultato in un file CSV
risultato = Settori_Medici.toPandas()
risultato.to_csv('/content/risultati/Settori_Medici.csv')

+------------------+---------------+--------------------+
|Settore_Medico    |Numero_Progetti|Somma_Finanziata_EUR|
+------------------+---------------+--------------------+
|Neurosciences     |60             |316032667           |
|Genetics          |146            |255874341           |
|Bioengineering    |44             |217362848           |
|Biotechnology     |99             |188532055           |
|Prevention        |55             |172584138           |
|Clinical Research |76             |158073322           |
|Nutrition         |34             |102271921           |
|Human Genome      |35             |98437210            |
|Rare Diseases     |72             |88654024            |
|Digestive Diseases|41             |76116470            |
+------------------+---------------+--------------------+



In [ ]:
!tar -cvf risultati.tar /content/risultati

tar: Removing leading `/' from member names
/content/risultati/
/content/risultati/Istituzioni.csv
/content/risultati/parolechiavi.csv
/content/risultati/capacitàfondi.csv
/content/risultati/ProgettiCancro.csv
/content/risultati/SommaFinanziataCancro.csv
/content/risultati/ProgettiFinanziatiSostenibilità.csv
/content/risultati/Top5DipSommaProgetti.csv
/content/risultati/SommaFinanziata.csv
/content/risultati/ricercadurata.csv
/content/risultati/ProgFondi2020.csv
/content/risultati/Tem_Top10.csv
/content/risultati/top10paesi.csv
/content/risultati/Dipartimenti.csv
/content/risultati/Settori_Medici.csv
